In [ ]:
# 🛠️ SETUP
!pip install PyPDF2 pandas nltk spacy gensim scikit-learn matplotlib seaborn -q
import nltk
nltk.download('stopwords')
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 757.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 869.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 74.8 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# 📚 IMPORTS
import os
import re
import PyPDF2
import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim import corpora, models
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

# 🔧 INIT
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Unzip the uploaded zip file (replace with your zip name)
!unzip ENG_PDF.zip -d gdpr_pdfs

Archive:  ENG_PDF.zip
  inflating: gdpr_pdfs/ETid-689.pdf  
  inflating: gdpr_pdfs/ETid-752.pdf  
  inflating: gdpr_pdfs/ETid-820.pdf  
  inflating: gdpr_pdfs/ETid-875.pdf  
  inflating: gdpr_pdfs/ETid-893.pdf  
  inflating: gdpr_pdfs/ETid-930.pdf  
  inflating: gdpr_pdfs/ETid-987.pdf  
  inflating: gdpr_pdfs/ETid-1009.pdf  
  inflating: gdpr_pdfs/ETid-1188.pdf  
  inflating: gdpr_pdfs/ETid-1190.pdf  
  inflating: gdpr_pdfs/ETid-1250.pdf  
  inflating: gdpr_pdfs/ETid-1373.pdf  
  inflating: gdpr_pdfs/ETid-1422.pdf  
  inflating: gdpr_pdfs/ETid-1461.pdf  
  inflating: gdpr_pdfs/ETid-1543.pdf  
  inflating: gdpr_pdfs/ETid-1564.pdf  
  inflating: gdpr_pdfs/ETid-1578.pdf  
  inflating: gdpr_pdfs/ETid-1666.pdf  
  inflating: gdpr_pdfs/ETid-1677.pdf  
  inflating: gdpr_pdfs/ETid-1696.pdf  
  inflating: gdpr_pdfs/ETid-1844.pdf  
  inflating: gdpr_pdfs/ETid-1965.pdf  
  inflating: gdpr_pdfs/ETid-2032.pdf  
  inflating: gdpr_pdfs/ETid-2317.pdf  
  inflating: gdpr_pdfs/ETid-2555.pdf  
  inflatin

In [ ]:
# 📥 1. Extract text from PDFs
def extract_text_from_pdfs(pdf_dir):
    texts = {}
    for filename in os.listdir(pdf_dir):
        if filename.endswith('.pdf'):
            with open(os.path.join(pdf_dir, filename), 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                text = " ".join(page.extract_text() for page in reader.pages)
                texts[filename] = text
    return texts

# Replace with your path
pdf_dir = 'gdpr_pdfs'  # Adjust if needed
texts = extract_text_from_pdfs(pdf_dir)
print(f"Extracted {len(texts)} PDF(s). Sample:\n{list(texts.keys())[:2]}")


[0, IndirectObject(1998, 0, 139398150638992)]
[0, IndirectObject(1949, 0, 139398150638992)]
[0, IndirectObject(1923, 0, 139398150638992)]
[0, IndirectObject(1897, 0, 139398150638992)]
[0, IndirectObject(1828, 0, 139398150638992)]
[0, IndirectObject(1812, 0, 139398150638992)]
[0, IndirectObject(1774, 0, 139398150638992)]
[0, IndirectObject(1498, 0, 139398150638992)]
[0, IndirectObject(3946, 0, 139401438805392)]
[0, IndirectObject(3851, 0, 139401438805392)]
[0, IndirectObject(3846, 0, 139401438805392)]
[0, IndirectObject(3841, 0, 139401438805392)]
[0, IndirectObject(3739, 0, 139401438805392)]
[0, IndirectObject(3676, 0, 139401438805392)]
[0, IndirectObject(3571, 0, 139401438805392)]
[0, IndirectObject(3495, 0, 139401438805392)]
[0, IndirectObject(3490, 0, 139401438805392)]
[0, IndirectObject(3389, 0, 139401438805392)]
[0, IndirectObject(2144, 0, 139401438805392)]
[0, IndirectObject(2353, 0, 139401438805392)]
[0, IndirectObject(2178, 0, 139401438805392)]
[0, IndirectObject(351, 0, 1393981

Extracted 33 PDF(s). Sample:
['ETid-1696.pdf', 'ETid-1373.pdf']


In [ ]:
# 🧼 2. Clean text and tokenize
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = [word for word in text.split() if word not in stop_words and len(word) > 3]
    return tokens


In [ ]:
# 🔎 3. Keyword frequency analysis
def keyword_frequency(text, keywords):
    return {kw: text.lower().count(kw) for kw in keywords}

keywords = ['monitoring', 'encryption', 'access control', 'destruct', 'logging', 'transfer',
            'privacy', 'rights', 'vulnerability', 'audit', 'personal data', 'breach', 'risk']

keyword_df = pd.DataFrame(
    [keyword_frequency(text, keywords) for text in texts.values()],
    index=texts.keys()
)

keyword_df.style.background_gradient(cmap='Blues')


,monitoring,encryption,access control,destruct,logging,transfer,privacy,rights,vulnerability,audit,personal data,breach,risk
ETid-1696.pdf,0,0,0,0,0,0,0,0,0,0,0,0,0
ETid-1373.pdf,0,0,0,0,0,0,4,57,1,0,48,13,68
ETid-930.pdf,3,1,0,3,0,3,1,8,1,1,37,56,19
ETid-2566.pdf,0,2,0,3,0,1,3,15,1,7,67,47,47
ETid-2561.pdf,0,0,0,0,0,0,0,0,0,0,0,0,0
ETid-2555.pdf,0,0,0,2,0,1,2,10,0,14,66,42,48
ETid-1578.pdf,3,1,0,1,0,3,54,36,0,0,166,20,15
ETid-2317.pdf,3,1,0,3,0,1,2,6,0,0,35,27,16
ETid-1461.pdf,2,1,2,5,1,1,3,12,19,1,82,32,32
ETid-1677.pdf,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 🧠 4. Topic modeling (LDA)
def perform_topic_modeling(texts, num_topics=6):
    tokenized_docs = [clean_text(text) for text in texts.values()]
    dictionary = corpora.Dictionary(tokenized_docs)
    corpus = [dictionary.doc2bow(text) for text in tokenized_docs]
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
    return lda_model.print_topics(), lda_model, corpus, dictionary

topics, lda_model, corpus, dictionary = perform_topic_modeling(texts)
for i, topic in topics:
    print(f"Topic {i}: {topic}")


Topic 0: 0.025*"data" + 0.018*"article" + 0.016*"whatsapp" + 0.016*"information" + 0.014*"processing" + 0.012*"gdpr" + 0.009*"facebook" + 0.008*"submissions" + 0.008*"user" + 0.008*"decision"
Topic 1: 0.027*"data" + 0.017*"meta" + 0.017*"article" + 0.015*"gdpr" + 0.009*"decision" + 0.009*"ireland" + 0.009*"edpb" + 0.007*"personal" + 0.006*"paragraph" + 0.006*"protection"
Topic 2: 0.034*"data" + 0.017*"personal" + 0.013*"article" + 0.010*"gdpr" + 0.010*"processing" + 0.008*"commissioner" + 0.007*"council" + 0.007*"breach" + 0.006*"security" + 0.006*"measures"
Topic 3: 0.021*"article" + 0.018*"data" + 0.013*"breach" + 0.012*"gdpr" + 0.009*"draft" + 0.009*"personal" + 0.009*"decision" + 0.008*"submissions" + 0.008*"controller" + 0.006*"processing"
Topic 4: 0.020*"processing" + 0.016*"article" + 0.016*"information" + 0.016*"users" + 0.016*"gdpr" + 0.014*"data" + 0.013*"instagram" + 0.013*"contact" + 0.012*"child" + 0.011*"account"
Topic 5: 0.002*"data" + 0.001*"gdpr" + 0.001*"article" + 0.

In [ ]:
# 📌 5. Named Entity Recognition (NER)
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['ORG', 'NUM', 'LAW']]

for filename, text in texts.items():
    entities = extract_entities(text)
    top_entities = pd.Series([ent[0] for ent in entities]).value_counts().head(10)
    print(f"\n📄 {filename} → {top_entities.to_dict()}")



📄 ETid-1696.pdf → {}

📄 ETid-1373.pdf → {'Meta  IE': 146, 'the IE SA': 106, 'LSA': 86, 'GDPR': 72, 'Instagram': 58, 'Article 6(1)(b': 49, 'Adopted': 49, 'NL SA': 45, 'Article 6(1)(f': 40, 'Meta IE': 39}

📄 ETid-930.pdf → {'the Cabinet Office': 38, 'ICO': 36, 'The Cabinet Office': 25, 'CSV': 24, 'GDPR': 24, 'Cabinet Office': 15, 'the Press Office': 13, 'Honours': 11, 'HAS': 9, 'DPA': 8}

📄 ETid-2566.pdf → {'DPP': 238, 'ICO': 32, 'the Cyber Incident': 22, 'DPA': 14, 'Tribunal': 12, 'the Cyber  Incident': 11, 'DPP Response': 10, 'Incident  Response': 6, 'SRA': 6, 'Article  33(1': 5}

📄 ETid-2561.pdf → {}

📄 ETid-2555.pdf → {'PSNI': 177, 'FOI': 87, 'ICO': 28, 'the Relevant Processing': 14, 'DPA': 14, 'CIB': 14, 'FOI Decision Maker': 10, 'PDF': 8, 'the FOI Service Instruction': 7, 'Article 5(1)(f': 7}

📄 ETid-1578.pdf → {'WhatsApp': 413, 'Commission': 275, 'Complaint': 118, 'GDPR': 94, 'Article 6(1)(b': 69, 'EU': 68, 'WhatsApp IE': 44, 'the Terms of Service': 41, 'Article 6(1': 41, 'the Ar

In [ ]:
import glob

# Adjust the folder path as needed
pdf_folder = "gdpr_pdfs"  # Update this!
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

In [ ]:
# Assign dominant topic to each document
dominant_topics = []
filenames = list(texts.keys())  # assuming 'texts' is a dict {filename: text}

for i, bow in enumerate(corpus):
    topic_dist = lda_model.get_document_topics(bow)
    top_topic = max(topic_dist, key=lambda x: x[1])  # (topic_id, confidence)
    dominant_topics.append({
        "Filename": os.path.basename(filenames[i]).replace(".pdf", ""),
        "Dominant_Topic": top_topic[0],
        "Topic_Confidence": top_topic[1]
    })

# Convert to DataFrame
doc_topics_df = pd.DataFrame(dominant_topics)
doc_topics_df.set_index("Filename", inplace=True)

# Show result
print(doc_topics_df)


           Dominant_Topic  Topic_Confidence
Filename                                   
ETid-1696               0          0.166667
ETid-1373               4          0.549971
ETid-930                2          0.855960
ETid-2566               2          0.972061
ETid-2561               0          0.166667
ETid-2555               1          0.793969
ETid-1578               0          0.671735
ETid-2317               2          0.999736
ETid-1461               2          0.999858
ETid-1677               0          0.166667
ETid-1666               2          0.652496
ETid-1190               2          0.997471
ETid-987                2          0.937653
ETid-752                2          0.999756
ETid-1543               4          0.936895
ETid-1422               2          0.986774
ETid-60                 3          0.820496
ETid-875                0          0.999960
ETid-1250               2          0.999764
ETid-1844               1          0.999211
ETid-570                2       